In [163]:
:dep graph-ds = { path = ".", default-features = false }
:dep anyhow = "1.0"
:dep h3o = "0.3.0"
// :dep plotters = "0.3.4"

In [164]:
use graph_ds::{Graph, Node, Edge, AStarResult};
use graph_ds::hexagon_graph::{cell_graph_from_mpk, h3_network_from_osm, h3_network_from_gtfs, cell::Cell, h3cell::H3Cell, osm::OSMLayer};

use std::time::Instant;
use std::sync::{Arc, RwLock, RwLockWriteGuard};
use std::collections::BTreeSet;
use std::collections::hash_map::DefaultHasher;
use std::hash::{Hash, Hasher};

use std::convert::TryInto;


## Cell Graph

In [165]:
let mut graph : Graph<Cell> = cell_graph_from_mpk("../resources/de_mirage_hexagons.mpk")?;
println!("Graph has {} nodes", graph.nr_nodes());

Graph has 7935 nodes


In [166]:
graph.node_hash()

7302240096543813149

In [167]:
fn heuristic(start_cell: &Cell, end_cell: &Cell) -> f64 {
    let dx = (start_cell.a - end_cell.a).abs();
    let dy = (start_cell.b - end_cell.b).abs();
    let dz = (start_cell.a + start_cell.b - end_cell.a - end_cell.b).abs();
    let dlayer = (start_cell.layer - end_cell.layer).abs();
    ((dx + dy + dz) as f64 / 2.0 + dlayer as f64) * (start_cell.radius * 2) as f64
}

In [168]:
let start = graph.get_random_node().unwrap();
let end = graph.get_random_node().unwrap();

println!("Start: {:?}", start);
println!("End: {:?}", end);

Start: Cell { a: -28, b: 9, radius: 24, layer: 0 }
End: Cell { a: -39, b: 39, radius: 24, layer: 0 }


In [178]:
let start = Cell { a: -43, b: 41, radius: 24, layer: 1 };
let end = Cell { a: 38, b: -9, radius: 24, layer: 0 };

In [179]:
println!("BFS start: {:?}, end: {:?}", start, end);
let now = Instant::now();
let (bfs_path, bfs_distance) = graph.bfs(start, Some(end), &None)?;
println!("time: {:?} µs", now.elapsed().as_micros());
println!("distance: {:?}", bfs_distance[0]);
println!("path length: {:?}", bfs_path);

println!("---");

println!("AStar start: {:?}, end: {:?}", start, end);
let now = Instant::now();
let astar_result = graph.astar(start, Some(end), &None, None, heuristic)?;
println!("time: {:?} µs", now.elapsed().as_micros());
println!("distance: {:?}", astar_result.distances.first().unwrap());
println!("path length: {:?}", astar_result.path.unwrap().len());

BFS start: Cell { a: -43, b: 41, radius: 24, layer: 1 }, end: Cell { a: 38, b: -9, radius: 24, layer: 0 }
time: 1013 µs
distance: None
path length: None
---
AStar start: Cell { a: -43, b: 41, radius: 24, layer: 1 }, end: Cell { a: 38, b: -9, radius: 24, layer: 0 }


no path found


## OSM + GTFS H3 Graph

In [171]:
let mut osm_g : Graph<H3Cell> = h3_network_from_osm("../resources/copenhagen-processed.osm.pbf", None)?;
println!("Graph has {} nodes, hash: {}", osm_g.nr_nodes(), osm_g.node_hash());

processing osm pbf file: ../resources/copenhagen-processed.osm.pbf
converted OSM file into 490098 edges
Graph has 418149 nodes, hash: 14980095458091112916


In [172]:
let mut gtfs_g : Graph<H3Cell> = h3_network_from_gtfs("../resources/rejseplanen.zip")?;
println!("Graph has {} nodes, hash: {}", gtfs_g.nr_nodes(), gtfs_g.node_hash());

getting GTFS feed from ../resources/rejseplanen.zip
routes: 1631
Graph has 103525 nodes, hash: 11658248065950047293


In [173]:
osm_g.merge(&mut gtfs_g)

Ok(())

In [174]:
let start = H3Cell {
    cell: 0x8c1f05902b4d1ff_u64.try_into().unwrap(),
    layer: -1,
};

let end = H3Cell {
    cell: 0x8c1f05902a717ff_u64.try_into().unwrap(),
    layer: -1,
};

fn h(start_cell: &H3Cell, end_cell: &H3Cell) -> f64 {
    start_cell
        .cell
        .grid_distance(end_cell.cell)
        .unwrap_or(i32::MAX) as f64
}

osm_g.astar(start, Some(end), &None, None, h)

[backtrace] found start node


Ok(AStarResult { path: Some([H3Cell { cell: 15-405440255150777 (8c1f05902b4d1ff), layer: -1 }, H3Cell { cell: 15-405440255150777 (8c1f05902b4d1ff), layer: 1606 }, H3Cell { cell: 15-405440251613777 (8c1f05902a717ff), layer: 1606 }, H3Cell { cell: 15-405440251613777 (8c1f05902a717ff), layer: -1 }]), single_target: true, distances: [24.333333333333332] })

In [175]:
osm_g.node_map.as_ref().read().unwrap().get_by_left(&start)

Some(459416)

In [176]:
// osm_g.edges.as_ref().read().unwrap().get(&439124)